<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/%E5%8A%B9%E6%9E%9C%E6%A4%9C%E8%A8%BC%E5%85%A5%E9%96%80_1_4%E3%83%A1%E3%83%BC%E3%83%AB%E3%83%9E%E3%83%BC%E3%82%B1%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E3%81%AE%E5%8A%B9%E6%9E%9C%E3%81%AE%E6%A4%9C%E8%A8%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

書籍「効果検証入門」の写経です。  
環境：GoogleColab  
言語：R  

In [16]:
# (2) ライブラリの読み出し
library("tidyverse")

#1.4.1 RCTを行ったデータの準備

In [17]:
# (3) データの読み込み
email_data <- read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

Parsed with column specification:
cols(
  recency = col_double(),
  history_segment = col_character(),
  history = col_double(),
  mens = col_double(),
  womens = col_double(),
  zip_code = col_character(),
  newbie = col_double(),
  channel = col_character(),
  segment = col_character(),
  visit = col_double(),
  conversion = col_double(),
  spend = col_double()
)



|  変数名  |  説明  |
| ---- | ---- |
|  recentry  |  最後の購入からの経過日数  |
|  history_segment  |  昨年の購入額の階層  |
|  history | 昨年の購入額  |
|mens|昨年に男性の商品を購入しているか?|
|womens|昨年に女性の商品を購入しているか?|
|zipcode|zipcodeをもとに地区を分類したもの|
|newbie|過去12カ月以内に新しくユーザになったか?|
|channel|昨年においてどのチャネルから購入したか?|
|segment|どのメールが配信されたか?｜
|visit|メールが配信されてから二週間以内にサイトへ訪問したか?|
|conversion|メールが配信されてから二週間以内に購入したか?|
|spend|購入した際の購入額|

In [18]:
# (4) データの準備
## 女性向けメールが配信されたデータを削除したデータを作成
male_df <- email_data %>%
  filter(segment != "Womens E-Mail") %>% # 女性向けメールが配信されたデータを削除
  mutate(treatment = ifelse(segment == "Mens E-Mail", 1, 0)) #介入を表すtreatment変数を追加

# 1.4.2 RCTデータの集計と有意差検定

In [19]:
# (5) 集計による比較
## group_byとsummairseを使って集計
male_df %>%
  group_by(treatment) %>% # データのグループ化
  summarise(conversion_rate = mean(conversion), # グループごとのconversionの平均
            spend_mean = mean(spend), # グループごとのspendの平均
            count = n()) # グループごとのデータ数

`summarise()` ungrouping output (override with `.groups` argument)



treatment,conversion_rate,spend_mean,count
<dbl>,<dbl>,<dbl>,<int>
0,0.005726087,0.6527894,21306
1,0.012531093,1.4226165,21307


In [20]:
# (6) t検定を行う
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail <- male_df %>%
  filter(treatment == 1) %>%
  pull(spend)

In [21]:
# (6) t検定を行う
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail <- male_df %>%
  filter(treatment == 1) %>%
  pull(spend)

## (b)メールが配信されなかったグループの購買データを得る
no_mail <- male_df %>%
  filter(treatment == 0) %>%
  pull(spend)

## (a)(b)の平均の差に対して有意差検定を実行する
t.test(mens_mail, no_mail, var.equal = T)


	Two Sample t-test

data:  mens_mail and no_mail
t = 5.3001, df = 42611, p-value = 1.163e-07
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.4851384 1.0545160
sample estimates:
mean of x mean of y 
1.4226165 0.6527894 


p値は有意水準5%を下回り、統計的に「mean of x」 と「mean of y」は 優位な差と言える。



---



---



# バイアスのあるデータによる効果の検証

In [22]:
# (7) セレクションバイアスのあるデータの作成
## seedを固定する
set.seed(1)

## 条件に反応するサンプルの量を半分にする
obs_rate_c <- 0.5
obs_rate_t <- 0.5

## バイアスのあるデータの作成
biased_data <- male_df %>%
  mutate(obs_rate_c =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), obs_rate_c, 1),
         obs_rate_t =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), 1, obs_rate_t),
         random_number = runif(n = NROW(male_df))) %>%
  filter( (treatment == 0 & random_number < obs_rate_c ) |
            (treatment == 1 & random_number < obs_rate_t) )


昨年の購買量が高いことや最近購入したユーザ、つまり潜在的に購入意欲が高そうなユーザに対してメールが多く配信されたデータを作り出した。

In [23]:
# (8) セレクションバイアスのあるデータで平均を比較
## group_byとsummairseを使って集計(Biased)
biased_data %>%
  group_by(treatment) %>%
  summarise(conversion_rate = mean(conversion),
            spend_mean = mean(spend),
            count = n())

`summarise()` ungrouping output (override with `.groups` argument)



treatment,conversion_rate,spend_mean,count
<dbl>,<dbl>,<dbl>,<int>
0,0.004977838,0.5483062,14665
1,0.013431794,1.5277526,17198


セレクションバイアスの結果により差が大きくなっている

In [24]:
# (9) Rの関数であるt.testを使ってt検定を行う(Biased)
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail_biased <- biased_data %>%
  filter(treatment == 1) %>%
  pull(spend)

## (b)メールが配信されなかったグループの購買データを得る
no_mail_biased <- biased_data %>%
  filter(treatment == 0) %>%
  pull(spend)

## (a)(b)の平均の差に対して有意差検定を実行
t.test(mens_mail_biased, no_mail_biased, var.equal = T)


	Two Sample t-test

data:  mens_mail_biased and no_mail_biased
t = 5.6708, df = 31861, p-value = 1.433e-08
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.6409145 1.3179784
sample estimates:
mean of x mean of y 
1.5277526 0.5483062 


p値はセレクションバイアスがない場合よりも小さくなっている。
ただし、分析の質が改善されたわけではない。  
統計的に有意な差があればどんな結果も正しい結果が推定できると考えることはできない。